<a href="http://datascience.disco.unimib.it/it/" target="_parent">
<img src="https://raw.githubusercontent.com/malborroni/Foundations_of_Computer-Science/master/images/DSunimib.png" alt="Open the site" width=95% align='left'/></a>


<br>
<br>

<a href="https://github.com/malborroni/RECMojion/blob/master/images/miscellaneous/Logo%206.png" target="_parent">
<img src="https://raw.githubusercontent.com/malborroni/RECMojion/master/images/miscellaneous/Logo%206.png" alt="Logo" width=35% align='center'/></a>

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <font color=894E97> Live Cam Detection with Classification </font>

<br>

***

> - __Authors__: &nbsp;&nbsp;Borroni Alessandro, Corvaglia Andre, Perletti Massimiliano
- __ID numbers__: &nbsp;&nbsp;800069, 802487, 847548
- __Academic year__: &nbsp;&nbsp;2019/2020
- __CdLM__: &nbsp;&nbsp;Data Science
- __E-mails__: &nbsp;&nbsp;a.borroni2@campus.unimib.it, a.corvaglia3@campus.unimib.it, m.perletti2@campus.unimib.it

***

## <font color=009DC6> 1. Import packages </font>

If you haven't done it yet, install this repository from GitHub:

In [1]:
#!pip install git+https://github.com/rcmalli/keras-vggface.git

In [2]:
import numpy as np
import os
import math
import time
import matplotlib.pyplot as plt

import cv2 as cv
from cv2 import VideoCapture as cap
from skimage import io

from keras.models import load_model
from keras_vggface.utils import preprocess_input

from utils import CFEVideoConf, image_resize

Using TensorFlow backend.


## <font color=009DC6> 2. Load the model and define the functions: </font>

In [3]:
#model = load_model('weights-senet50-full2.best.hdf5')
#print('Loaded model from disk')

mod1 = load_model('preproc_final.h5')
#mod2 = load_model('pre_optimization.h5')
mod2 = load_model('opt_best.h5')
print('Both models are succesfully loaded')

C:\Users\MioPC\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Both models are succesfully loaded


In [4]:
def labelling(img, model):
    #-- fase di preprocess per l'immagine (to vggface input)
    img = np.expand_dims(img, axis = 0)
    x = img.astype('float32')
    x = preprocess_input(x, version = 2)
    
    #-- predict sul modello pre-trainato
    predicted = mod1.predict(x)
    #tmp = mod1.predict(x)
    predicted = model.predict(predicted)
    
    #if np.max(predicted) >= .5:
    #-- acquiszione valore di accuracy
    acc = round(np.max(predicted), 3)
        
    #-- selezione per il labelling
    pred = np.argmax(predicted, axis = 1)
    if pred == 0:
        label = 'Angry'
        emoji = cv.imread("images/emoji/angry_mod.png", -1)
    if pred == 1:
        label = 'Disgust'
        emoji = cv.imread("images/emoji/disgust_mod.png", -1)
    if pred == 2:
        label = 'Fear'
        emoji = cv.imread("images/emoji/fear_mod.png", -1)
    if pred == 3:
        label = 'Happy'
        emoji = cv.imread("images/emoji/happy_mod.png", -1)
    if pred == 4:
        label = 'Neutral'
        emoji = cv.imread("images/emoji/neutral_mod.png", -1)
    if pred == 5:
        label = 'Sad'
        emoji = cv.imread("images/emoji/sad_mod.png", -1)
    if pred == 6:
        label = 'Surprise'
        emoji = cv.imread("images/emoji/surprise_mod.png", -1)
#     else:
#         label = 'Unknown'

    return label, acc, emoji

In [5]:
def detect(image, model, classifier,
           scale_factor = 1.1, minNeighbors = 5,
           size = (224,224)):
    '''
    This function takes in input images and after face detection apply a crop and a resize to that images.
    
    @params:
    from_directory = where to find the photos
    to_directory = where to save the photos
    classifier = face detector pre-trained (XML file)
    size = tuple
    color = RGB or BW
    init_number = first number to start naming photos
    '''
    face_cascade = cv.CascadeClassifier(classifier)
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scale_factor, minNeighbors)

    if len(faces) == 1:
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+h] # rec
            roi_color = image[y:y+h, x:x+w]
            n_img = cv.resize(roi_color, size)
        
            #-- color B, G, R
            
            #color_face = (80,130,0)
            #rectangle_bgr = (80,130,0)
            
            #color_face = (24,191,255) #oro
            color_text = (255,255,255) #bianco
            #rectangle_bgr = (24,191,255) #oro
            #color_face = (255,204,0) #azzurro
            #color_text = (0,153,255) #arancione
            #rectangle_bgr = (255,204,0) #arancione
            
            #-- setup font e size per il testo
            font = cv.FONT_HERSHEY_TRIPLEX
            #font = cv.FONT_HERSHEY_DUPLEX
            font_scale = 0.88
            
            #-- rettangolo face detection
            #cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            #QUI
            
            #-- labelling (con model)
            label, acc, emoji = labelling(n_img, model)
            #-- preparo la stringa da stampare
            string = " " + label + ": " + str(acc) + " "
            
            if label == 'Angry':
                color_face = (0,32,225)
                rectangle_bgr = (0,32,225)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            if label == 'Disgust':
                color_face = (0,66,96)
                rectangle_bgr = (0,66,96)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            if label == 'Fear':
                color_face = (95,0,64)
                rectangle_bgr = (95,0,64)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            if label == 'Happy':
                color_face = (0,128,0)
                rectangle_bgr = (0,128,0)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            if label == 'Neutral':
                color_face = (94,96,55)
                rectangle_bgr = (94,96,55)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            if label == 'Sad':
                color_face = (128,47,0)
                rectangle_bgr = (128,47,0)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            if label == 'Surprise':
                color_face = (24,191,255)
                rectangle_bgr = (24,191,255)
                cv.rectangle(image, (x,y), (x+w,y+h), color_face, 3)
            
            #-- rettangolo per background stringa
            (text_width, text_height) = cv.getTextSize(string, font, fontScale=font_scale, thickness=1)[0]
            box_coords = ((x-2, y), (x + text_width + 4, y - text_height - 8))
            cv.rectangle(image, box_coords[0], box_coords[1], rectangle_bgr, cv.FILLED)
            #-- inserimento stringa
            cv.putText(image, string, (x,y-5), font, font_scale, color_text, 1)
            
            for (ex, ey, ew, eh) in faces:
                roi_face = roi_gray[ey: ey + eh, ex: ex + ew]
                emojis = image_resize(emoji.copy(), width = w//4)
                
                #-- Salvo valori shape di "faces" 
                faces_h, faces_w = faces.shape
                
                gw, gh, gc = emojis.shape
                
                for i in range(0, gw):
                    for j in range(0, gh):
                        #print(emoji[i, j]) #RGBA
                        if emojis[i, j][3] != 0: # alpha 0
                            offset = 10
                            h_offset = faces_h - gh - offset
                            w_offset = faces_w - gw - offset
                            roi_color[h_offset + i, w_offset + j] = emojis[i, j, :-1]
            
    else:
        pass

In [6]:
def live_cam(model, classifier, resolution = (1280, 720), fps = 30, time_sleep = 2):
    '''
    This function allow to access to pc webcam in order to acquire images of the subject.
    
    @params:
    directory = where to save the captured images
    subject = name of the person who is in photo
    init_number = first number to start naming photos
    counter_limit = if desired, limit the number of photos captured
    resolution = (width, height)
    fps = Frame Per Second
    time_sleep = sleep between two photos
    numbered = Bool, necessary in order to apply counter_limit
    '''
    cam = cv.VideoCapture(0) #maybe change here

    cam.set(cv.CAP_PROP_FRAME_WIDTH, resolution[0])
    cam.set(cv.CAP_PROP_FRAME_HEIGHT, resolution[1])
    cam.set(cv.CAP_PROP_FPS, fps)

    cv.namedWindow("Acquisition window")

    while True:
        ret, frame = cam.read()
        if not ret:
            break
        k = cv.waitKey(1)

        time.sleep(time_sleep)

        detect(frame, model, classifier=classifier, scale_factor = 1.1, minNeighbors = 6)
        
        text = 'Press ESC to exit from demo...'
        cv.putText(frame, text, (10,20), cv.FONT_HERSHEY_DUPLEX, 0.75, (255,255,255), 1)
        
        cv.imshow("Acquisition window", frame)

        
        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        

    cam.release()
    cv.destroyAllWindows()

## <font color=009DC6> 3. Launch the Live Demo </font>

In order to interrupt the video recording, press "ESC" on the keyboard.

In [7]:
live_cam(time_sleep = 0.01, model = mod2, classifier = 'haarcascade_frontalface_alt2.xml')

Escape hit, closing...


&nbsp;